# Intro to Ray Data

## Ray Datasets: Distributed Data Preprocessing

Ray Datasets are the standard way to load and exchange data in Ray libraries and applications. They provide basic distributed data transformations such as maps ([`map_batches`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html#ray.data.Dataset.map_batches "ray.data.Dataset.map_batches")), global and grouped aggregations ([`GroupedDataset`](https://docs.ray.io/en/latest/data/api/doc/ray.data.grouped_dataset.GroupedDataset.html#ray.data.grouped_dataset.GroupedDataset "ray.data.grouped_dataset.GroupedDataset")), and shuffling operations ([`random_shuffle`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.random_shuffle.html#ray.data.Dataset.random_shuffle "ray.data.Dataset.random_shuffle"), [`sort`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.sort.html#ray.data.Dataset.sort "ray.data.Dataset.sort"), [`repartition`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.repartition.html#ray.data.Dataset.repartition "ray.data.Dataset.repartition")), and are compatible with a variety of file formats, data sources, and distributed frameworks.

Here's an overview of the integrations with other processing frameworks, file formats, and supported operations, as well as a glimpse at the Ray Datasets API.

Check the [Input/Output reference](https://docs.ray.io/en/latest/data/api/input_output.html#input-output) to see if your favorite format is already supported.

<img src='https://docs.ray.io/en/releases-2.6.1/_images/dataset.svg' width=80%/>

### Datasets for Parallel Compute
-------------------------------------------------------------------------------------------------------------------------------------------

Datasets also simplify general purpose parallel GPU and CPU compute in Ray; for instance, for [GPU batch inference](https://docs.ray.io/en/latest/ray-overview/use-cases.html#ref-use-cases-batch-infer). They provide a higher-level API for Ray tasks and actors for such embarrassingly parallel compute, internally handling operations like batching, pipelining, and memory management.

<img src='https://docs.ray.io/en/releases-2.6.1/_images/stream-example.png' width=60%/>

As part of the Ray ecosystem, Ray Datasets can leverage the full functionality of Ray's distributed scheduler, e.g., using actors for optimizing setup time and GPU scheduling.

### Reading Data[](https://docs.ray.io/en/latest/data/key-concepts.html#reading-data "Permalink to this headline")

Datasets uses Ray tasks to read data from remote storage. When reading from a file-based datasource (e.g., S3, GCS), it creates a number of read tasks proportional to the number of CPUs in the cluster. Each read task reads its assigned files and produces an output block:

<img src='https://docs.ray.io/en/releases-2.6.1/_images/dataset-read.svg' width=60%/>

In [1]:
import ray

ds = ray.data.read_csv("s3://anyscale-public-materials/ray-ai-libraries/Iris.csv")

ds

2026-01-20 20:26:53,722	INFO worker.py:1821 -- Connecting to existing Ray cluster at address: 10.0.142.230:6379...
2026-01-20 20:26:53,733	INFO worker.py:1998 -- Connected to Ray cluster. View the dashboard at https://session-v4klp1kjtnk9yrxwdcz5ah11ub.i.anyscaleuserdata.com 
2026-01-20 20:26:53,758	INFO packaging.py:463 -- Pushing file package 'gcs://_ray_pkg_0b7f0c167e67268a4d6bf905ca9322f25f293996.zip' (9.27MiB) to Ray cluster...
2026-01-20 20:26:53,795	INFO packaging.py:476 -- Successfully pushed file package 'gcs://_ray_pkg_0b7f0c167e67268a4d6bf905ca9322f25f293996.zip'.
/home/ray/anaconda3/lib/python3.12/site-packages/ray/_private/worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Dataset(num_rows=?, schema=Unknown schema)

In [2]:
ds.show(3)

2026-01-20 20:27:01,141	INFO dataset.py:3641 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2026-01-20 20:27:01,146	INFO logging.py:397 -- Registered dataset logger for dataset dataset_166_0
2026-01-20 20:27:01,167	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_166_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:27:01,167	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_166_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> LimitOperator[limit=3]
2026-01-20 20:27:01,168	INFO streaming_executor.py:687 -- [dataset]: A new progress UI is available. To enable, set `ray.data.DataContext.get_current().enable_rich_progress_bars = True` and `ray.data.DataContext.get_current().use_ray_tqdm = False`.
2026-01-20 20:27:01,169	INFO progress_bar.py:155 -- Progress bar disabled because stdout is a non-int

(autoscaler +1m40s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


2026-01-20 20:28:34,990	INFO progress_bar.py:215 -- ListFiles: Tasks: 0; Actors: 0; Queued blocks: 0 (0.0B); Resources: 0.0 CPU, 0.0B object store: Progress Completed 1 / 1
2026-01-20 20:28:34,991	INFO progress_bar.py:215 -- ReadFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 16.8KiB object store: Progress Completed 150 / ?
2026-01-20 20:28:34,992	INFO progress_bar.py:215 -- limit=3: Tasks: 0; Actors: 0; Queued blocks: 0 (0.0B); Resources: 0.0 CPU, 0.0B object store: Progress Completed 0 / ?
2026-01-20 20:28:34,992	INFO progress_bar.py:215 -- Running Dataset: dataset_166_0. Active & requested resources: 1/32 CPU, 25.3KiB/26.9GiB object store: Progress Completed 0 / ?
2026-01-20 20:28:35,001	INFO streaming_executor.py:305 -- ✔️  Dataset dataset_166_0 execution finished in 93.83 seconds
INFO:openlineage.client.client:OpenLineageClient will use `composite` transport
INFO:openlineage.client.transport.composite:Stopping OpenLineage CompositeTransport emission after 

{'Id': 1, 'SepalLengthCm': 5.1, 'SepalWidthCm': 3.5, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa'}
{'Id': 2, 'SepalLengthCm': 4.9, 'SepalWidthCm': 3.0, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa'}
{'Id': 3, 'SepalLengthCm': 4.7, 'SepalWidthCm': 3.2, 'PetalLengthCm': 1.3, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa'}


In [3]:
ds.write_parquet('/mnt/cluster_storage/parquet_iris')

2026-01-20 20:28:35,103	INFO logging.py:397 -- Registered dataset logger for dataset dataset_168_0
2026-01-20 20:28:35,107	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_168_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:28:35,108	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_168_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Write]
2026-01-20 20:28:35,125	INFO progress_bar.py:213 -- === Ray Data Progress {ListFiles} ===
2026-01-20 20:28:35,126	INFO progress_bar.py:215 -- ListFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 384.0MiB object store: Progress Completed 0 / ?
2026-01-20 20:28:35,126	INFO progress_bar.py:213 -- === Ray Data Progress {ReadFiles} ===
2026-01-20 20:28:35,127	INFO progress_bar.py:215 -- ReadFiles: Tasks: 0; Actors: 0; Queued blocks: 0 (0.0B); Resources: 0.0 CPU, 0.0B object 

<div class="alert alert-block alert-warning">
<b>Note</b> 

We  write to `/mnt/cluster_storage`. This is a path that is available on all nodes in the cluster. If instead you use a path that is only local to one of the nodes in a multi-node cluster, you will see errors like `FileNotFoundError: [Errno 2] No such file or directory: '/path/to/file'`.

This is because Ray Data is designed to work with distributed storage systems like S3, HDFS, etc. If you want to write to local storage, you can add a special prefix `local://` to the path. In this case, Ray will only schedule tasks on the head node of the cluster.
</div>

In [4]:
! ls -l /mnt/cluster_storage/parquet_iris/

total 8
-rw-r--r-- 1 ray users 3722 Jan 20 18:30 0_1679641afb914a4d941be4a3b7a7cf3c_000000_000000-0.parquet
-rw-r--r-- 1 ray users 3722 Jan 20 20:28 164_953e278c878a459e80c7f9a52b83212b_000000_000000-0.parquet


### Transforming Data[](https://docs.ray.io/en/latest/data/key-concepts.html#transforming-data "Permalink to this headline")

Datasets can use either Ray tasks or Ray actors to transform datasets. Using actors allows for expensive state initialization (e.g., for GPU-based tasks) to be cached.

In [5]:
ds = ds.repartition(5)

ds

Repartition
+- Dataset(num_rows=?, schema=Unknown schema)

In [6]:
ds.take_batch(5)

2026-01-20 20:28:55,537	INFO logging.py:397 -- Registered dataset logger for dataset dataset_172_0
2026-01-20 20:28:55,542	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_172_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:28:55,543	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_172_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> AllToAllOperator[Repartition] -> LimitOperator[limit=5]
2026-01-20 20:28:55,562	INFO progress_bar.py:213 -- === Ray Data Progress {ListFiles} ===
2026-01-20 20:28:55,563	INFO progress_bar.py:215 -- ListFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 384.0MiB object store: Progress Completed 0 / ?
2026-01-20 20:28:55,563	INFO progress_bar.py:213 -- === Ray Data Progress {ReadFiles} ===
2026-01-20 20:28:55,564	INFO progress_bar.py:215 -- ReadFiles: Tasks: 0; Actors: 0; Queued blocks: 0 (0.0B); Res

{'Id': array([1, 2, 3, 4, 5]),
 'SepalLengthCm': array([5.1, 4.9, 4.7, 4.6, 5. ]),
 'SepalWidthCm': array([3.5, 3. , 3.2, 3.1, 3.6]),
 'PetalLengthCm': array([1.4, 1.4, 1.3, 1.5, 1.4]),
 'PetalWidthCm': array([0.2, 0.2, 0.2, 0.2, 0.2]),
 'Species': array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa'], dtype=object)}

### Shuffling Data[](https://docs.ray.io/en/latest/data/key-concepts.html#shuffling-data "Permalink to this headline")

Certain operations like *sort* or *groupby* require data blocks to be partitioned by value, or *shuffled*. Datasets uses tasks to implement distributed shuffles in a map-reduce style, using map tasks to partition blocks by value, and then reduce tasks to merge co-partitioned blocks together.

You can also change just the number of blocks of a Dataset using [`repartition()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.repartition.html#ray.data.Dataset.repartition "ray.data.Dataset.repartition"). Repartition has two modes:

1.  `shuffle=False` - performs the minimal data movement needed to equalize block sizes __(the default)__

2.  `shuffle=True` - performs a full distributed shuffle __(not the default because it's more expensive)__

<img src='https://docs.ray.io/en/releases-2.6.1/_images/dataset-shuffle.svg' width=60%/>

Datasets shuffle can scale to processing hundreds of terabytes of data. See the [Performance Tips Guide](https://docs.ray.io/en/latest/data/performance-tips.html#shuffle-performance-tips) for an in-depth guide on shuffle performance.

In [9]:
def transform_batch(batch):
    areas = []
    for ix in range(len(batch['Id'])):
        areas.append(batch["PetalLengthCm"][ix] * batch["PetalWidthCm"][ix])        
    batch['approximate_petal_area'] = areas
    return batch

In [10]:
ds.map_batches(transform_batch) \
  .take_batch()

2026-01-20 20:31:45,921	INFO logging.py:397 -- Registered dataset logger for dataset dataset_178_0
2026-01-20 20:31:45,925	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_178_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:31:45,926	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_178_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> AllToAllOperator[Repartition] -> LimitOperator[limit=20] -> TaskPoolMapOperator[MapBatches(transform_batch)]
2026-01-20 20:31:45,943	INFO progress_bar.py:213 -- === Ray Data Progress {ListFiles} ===
2026-01-20 20:31:45,944	INFO progress_bar.py:215 -- ListFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 384.0MiB object store: Progress Completed 0 / ?
2026-01-20 20:31:45,945	INFO progress_bar.py:213 -- === Ray Data Progress {ReadFiles} ===
2026-01-20 20:31:45,946	INFO progress_bar.py:215 -- ReadFil

{'Id': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20]),
 'SepalLengthCm': array([5.1, 4.9, 4.7, 4.6, 5. , 5.4, 4.6, 5. , 4.4, 4.9, 5.4, 4.8, 4.8,
        4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1]),
 'SepalWidthCm': array([3.5, 3. , 3.2, 3.1, 3.6, 3.9, 3.4, 3.4, 2.9, 3.1, 3.7, 3.4, 3. ,
        3. , 4. , 4.4, 3.9, 3.5, 3.8, 3.8]),
 'PetalLengthCm': array([1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.4, 1.5, 1.4, 1.5, 1.5, 1.6, 1.4,
        1.1, 1.2, 1.5, 1.3, 1.4, 1.7, 1.5]),
 'PetalWidthCm': array([0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2, 0.2, 0.1, 0.2, 0.2, 0.1,
        0.1, 0.2, 0.4, 0.4, 0.3, 0.3, 0.3]),
 'Species': array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
        'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa'],
      

### More example operations: transforming Datasets

Datasets transformations take in datasets and produce new datasets. For example, *map* is a transformation that applies a user-defined function on each dataset record and returns a new dataset as the result. Datasets transformations can be composed to express a chain of computations.

There are two main types of transformations:

-   One-to-one: each input block will contribute to only one output block, such as [`ds.map_batches()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html#ray.data.Dataset.map_batches "ray.data.Dataset.map_batches").

-   All-to-all: input blocks can contribute to multiple output blocks, such as [`ds.random_shuffle()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.random_shuffle.html#ray.data.Dataset.random_shuffle "ray.data.Dataset.random_shuffle").

Here is a table listing some common transformations supported by Ray Datasets.

Common Ray Datasets transformations.[](https://docs.ray.io/en/latest/data/transforming-datasets.html#id2 "Permalink to this table")

| Transformation | Type | Description |
| --- | --- | --- |
|[`ds.map_batches()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html#ray.data.Dataset.map_batches "ray.data.Dataset.map_batches")|One-to-one|Apply a given function to batches of records of this dataset.|
|[`ds.add_column()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.add_column.html#ray.data.Dataset.add_column "ray.data.Dataset.add_column")|One-to-one|Apply a given function to batches of records to create a new column.|
|[`ds.drop_columns()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.add_column.html#ray.data.Dataset.add_column "ray.data.Dataset.add_column")|One-to-one|Drop the given columns from the dataset.|
|[`ds.split()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.split.html#ray.data.Dataset.split "ray.data.Dataset.split")|One-to-one|Split the dataset into N disjoint pieces.|
|[`ds.repartition(shuffle=False)`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.repartition.html#ray.data.Dataset.repartition "ray.data.Dataset.repartition")|One-to-one|Repartition the dataset into N blocks, without shuffling the data.|
|[`ds.repartition(shuffle=True)`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.repartition.html#ray.data.Dataset.repartition "ray.data.Dataset.repartition")|All-to-all|Repartition the dataset into N blocks, shuffling the data during repartition.|
|[`ds.random_shuffle()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.random_shuffle.html#ray.data.Dataset.random_shuffle "ray.data.Dataset.random_shuffle")|All-to-all|Randomly shuffle the elements of this dataset.|
|[`ds.sort()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.sort.html#ray.data.Dataset.sort "ray.data.Dataset.sort")|All-to-all|Sort the dataset by a sortkey.|
|[`ds.groupby()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.groupby.html#ray.data.Dataset.groupby "ray.data.Dataset.groupby")|All-to-all|Group the dataset by a groupkey.|

### Execution mode

Most transformations are **lazy** in Ray Data - i.e. they don't execute until you either:
- **write a dataset to storage**
- explicitly **materialize** the data
- **iterate over the dataset** (usually when feeding data to model training).

To explicitly *materialize* a very small subset of the data, you can use the `take_batch` method.
Most transformations are lazy. They don't execute until you consume a dataset or call [`Dataset.materialize()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.materialize.html#ray.data.Dataset.materialize "ray.data.Dataset.materialize").

The transformations are executed in a streaming way, incrementally on the data and with operators processed in parallel. For an in-depth guide on Datasets execution, read https://docs.ray.io/en/releases-2.8.0/data/data-internals.html

> To force computation and local caching of the entire dataset, you can used `materialize`. Consider the performance constraints and impacts of caching, though

In [11]:
ds.materialize()

2026-01-20 20:33:15,588	INFO logging.py:397 -- Registered dataset logger for dataset dataset_179_0
2026-01-20 20:33:15,591	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_179_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:33:15,592	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_179_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> AllToAllOperator[Repartition]
2026-01-20 20:33:15,609	INFO progress_bar.py:213 -- === Ray Data Progress {ListFiles} ===
2026-01-20 20:33:15,609	INFO progress_bar.py:215 -- ListFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 384.0MiB object store: Progress Completed 0 / ?
2026-01-20 20:33:15,610	INFO progress_bar.py:213 -- === Ray Data Progress {ReadFiles} ===
2026-01-20 20:33:15,611	INFO progress_bar.py:215 -- ReadFiles: Tasks: 0; Actors: 0; Queued blocks: 0 (0.0B); Resources: 0.0 CPU, 0.0B obje

MaterializedDataset(
   num_blocks=5,
   num_rows=150,
   schema={
      Id: int64,
      SepalLengthCm: double,
      SepalWidthCm: double,
      PetalLengthCm: double,
      PetalWidthCm: double,
      Species: string
   }
)

### Transforming data with actors

When using the actor compute strategy, per-row and per-batch UDFs can also be *callable classes*, i.e. classes that implement the `__call__` magic method. The constructor of the class can be used for stateful setup, and will be only invoked once per worker actor.

To implement this, you can use the `map_batches` API with a "Callable" class method that implements:

- `__init__`: Initialize any expensive state.
- `__call__`: Perform the stateful transformation.

<div class="alert alert-block alert-warning">
<b>Note:</b> These transformation APIs take the uninstantiated callable class as an argument, not an instance of the class.
</div>

In [14]:
class ModelExample:
    def __init__(self):
        expensive_model_weights = [ 0.3, 1.75 ]
        self.complex_model = lambda petal_width: (petal_width + expensive_model_weights[0])  ** expensive_model_weights[1]

    def __call__(self, batch):
        batch["predictions"] = self.complex_model(batch["PetalWidthCm"])
        return batch

ds.map_batches(ModelExample, 
               compute=ray.data.ActorPoolStrategy(size=2)) \
  .show(10)

2026-01-20 20:35:13,500	INFO logging.py:397 -- Registered dataset logger for dataset dataset_182_0
2026-01-20 20:35:13,505	INFO streaming_executor.py:178 -- Starting execution of Dataset dataset_182_0. Full logs are in /tmp/ray/session_2026-01-20_18-18-31_241199_2386/logs/ray-data
2026-01-20 20:35:13,506	INFO streaming_executor.py:179 -- Execution plan of Dataset dataset_182_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> AllToAllOperator[Repartition] -> LimitOperator[limit=10] -> ActorPoolMapOperator[MapBatches(ModelExample)]
2026-01-20 20:35:13,669	INFO progress_bar.py:213 -- === Ray Data Progress {ListFiles} ===
2026-01-20 20:35:13,670	INFO progress_bar.py:215 -- ListFiles: Tasks: 1; Actors: 0; Queued blocks: 0 (0.0B); Resources: 1.0 CPU, 384.0MiB object store: Progress Completed 0 / ?
2026-01-20 20:35:13,671	INFO progress_bar.py:213 -- === Ray Data Progress {ReadFiles} ===
2026-01-20 20:35:13,672	INFO progress_bar.py:215 -- ReadFiles

{'Id': 1, 'SepalLengthCm': 5.1, 'SepalWidthCm': 3.5, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa', 'predictions': 0.29730177875068026}
{'Id': 2, 'SepalLengthCm': 4.9, 'SepalWidthCm': 3.0, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa', 'predictions': 0.29730177875068026}
{'Id': 3, 'SepalLengthCm': 4.7, 'SepalWidthCm': 3.2, 'PetalLengthCm': 1.3, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa', 'predictions': 0.29730177875068026}
{'Id': 4, 'SepalLengthCm': 4.6, 'SepalWidthCm': 3.1, 'PetalLengthCm': 1.5, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa', 'predictions': 0.29730177875068026}
{'Id': 5, 'SepalLengthCm': 5.0, 'SepalWidthCm': 3.6, 'PetalLengthCm': 1.4, 'PetalWidthCm': 0.2, 'Species': 'Iris-setosa', 'predictions': 0.29730177875068026}
{'Id': 6, 'SepalLengthCm': 5.4, 'SepalWidthCm': 3.9, 'PetalLengthCm': 1.7, 'PetalWidthCm': 0.4, 'Species': 'Iris-setosa', 'predictions': 0.5356999058252557}
{'Id': 7, 'SepalLengthCm': 4.6, 'SepalWidthCm': 3.4, 

### Dataset details

A Dataset consists of a list of Ray object references to *blocks*. Having multiple blocks in a dataset allows for parallel transformation and ingest.

The following figure visualizes a tabular dataset with three blocks, each block holding 1000 rows each:

|<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-intro/block.png" width="700px" loading="lazy">|
|:--|
|A Dataset when materialized is a distributed collection of blocks. This example illustrates a materialized dataset with three blocks, each block holding 1000 rows.|

Since a Dataset is just a list of Ray object references, it can be freely passed between Ray tasks, actors, and libraries like any other object reference. This flexibility is a unique characteristic of Ray Datasets.

<div class="alert alert-block alert-success">
    
__Lab activity: Stateless transformation__
    
1. Create a Ray Dataset from the iris data in `/mnt/cluster_storage/parquet_iris`
1. Create a "sum of features" transformation that calculates the sum of the Sepal Length, Sepal Width, Petal Length, and Petal Width features for the records
    1. Design this transformation to take a Ray Dataset *batch* of records
    1. Return the records without the ID column but with an additional column called "sum"
    1. Hint: you do not need to use NumPy, but the calculation may be easier/simpler to code using NumPy vectorized operations with the records in the batch
</div>


<div class="alert alert-block alert-warning">
    
__Lab activity: Stateful transformation__
    
1. Create a Ray Dataset from the iris data in `/mnt/cluster_storage/parquet_iris`
1. Create an class that makes predictions on iris records using these steps:
    1. in the class constructor, create an instance of the following "model" class:
        ```python

          class SillyModel():

              def predict(self, petal_length):
                  return petal_length + 0.42


        ```
    1. in the `__call__` method of the actor class
        1. take a batch of records
        1. create predictions for each record in the batch using the model instance
            1. Hint: the code may be simpler using NumPy vectorized operations
        1. add the predictions to the record batch
        1. return the new, augmented batch
1. Use that class to perform batch inference on the dataset using actors
</div>